In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import scipy.stats as scs
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV, cross_val_score
import xgboost as xgb

In [ ]:
# Data from: https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe

In [ ]:
data = pd.read_csv('Hotel_Reviews.csv')

# remove rows with no reviews
data = data[(data['Negative_Review'] != 'No Negative') | (data['Positive_Review'] != 'No Positive')]
data.reset_index(drop=True, inplace=True)

In [ ]:
data.info()

In [ ]:
#how many unqiue in each column
#for i in data.columns:
#    print(i, len(data[i].unique()))

In [ ]:
#Drop unwanted columns
data.drop(['Hotel_Name','Hotel_Address','Review_Date','days_since_review'],  axis=1, inplace=True)
data['Average_Score'].astype(float)

In [ ]:
# Benchmark - error if you always guessed the average (1.18)
mean_absolute_error(data['Average_Score'], data['Reviewer_Score'])

In [ ]:
#correcting tags from 55,000 to a few - started as a list of strings that looked like lists of strings
#tags = data.Tags

#import ast
#new_tags = []
#for i in tags:
#    new_tags.append(ast.literal_eval(i))

# check most common tags
from collections import Counter
list_for_counting = []
for i in new_tags:
    for t in i:
        list_for_counting.append(t)
    
c = Counter(list_for_counting)
c.most_common(20)      

In [ ]:
'''# feature engineering tags to reduce columns
for index, i in enumerate(new_tags):
    for idx, t in enumerate(i):
        if t not in ['Stayed 1-2 nights','Stayed 3-4 nights', 'Stayed 5+ nights', 'Fancy', 'Budget', 'Medium', 'High', ' Business trip ', ' Solo traveler ', ' Leisure trip ', ' Couple ', ' Group ', ' Family with young children ', ' Family with older children ']:
            if t in [' Stayed 1 night ',' Stayed 2 nights ']:
                new_tags[index][idx] = 'Stayed 1-2 nights'
            if t in [' Stayed 3 nights ',' Stayed 4 nights ']:
                new_tags[index][idx] = 'Stayed 3-4 nights'
            if t in [' Stayed 5 nights ',' Stayed 6 nights ', ' Stayed 7 nights ', ' Stayed 8 nights ', ' Stayed 9 nights ', ' Stayed 10 nights ',  ' Stayed 11 nights ',
                 ' Stayed 12 nights ', ' Stayed 13 nights ', ' Stayed 14 nights ', ' Stayed 15 nights ', ' Stayed 16 nights ', ' Stayed 17 nights ',' Stayed 18 nights ', ' Stayed 19 nights ', ' Stayed 20 nights ',
                 ' Stayed 21 nights ', ' Stayed 22 nights ', ' Stayed 23 nights ', ' Stayed 24 nights ', ' Stayed 25 nights ', ' Stayed 26 nights ',
                 ' Stayed 27 nights ', ' Stayed 28 nights ', ' Stayed 29 nights ', ' Stayed 30 nights ', ' Stayed 31 nights ',]:
                new_tags[index][idx] = 'Stayed 5+ nights'
            if 'Luxury' in t or 'VIP' in t or 'Executive' in t or 'Ambassador' in t or 'Royal' in t or 'Penthouse' in t or 'Suite' in t or 'Duplex' in t or 'Presidential' in t or 'Apartment' in t or 'Apartement' in t:
                new_tags[index][idx] = 'Fancy'
            if 'Superior' in t or 'Premium' in t or 'Prestige' in t or 'Premiere' in t or 'Privilege' in t or 'Deluxe' in t or 'Premier' in t or 'Club' in t or 'View' in t or 'Art' in t or 'Fabulous' in t or 'Wonderful' in t or 'Loft' in t or 'Eiffel' in t or 'Spa' in t or 'King' in t:
                new_tags[index][idx] = 'High'
            if 'Standard' in t or 'Budget' in t or 'Small' in t or 'Economy' in t or 'Basic' in t or 'Bunk Bed' in t or 'Interior' in t or 'Special Offer' in t or 'Triple' in t or 'Quadruple' in t or 'Quintuple' in t or 'Sextuple' in t or 'Junior' in t or 'Twin' in t or 'Mini' in t or 'Check In' in t or 'Check in' in t or'Solo' in t or 'Camper' in t or 'Rooms' in t or 'Interconnecting' in t or 'FAMILY' in t or 'Atrium' in t or 'rooms' in t:
                new_tags[index][idx] = 'Budget'
            if 'Comfort' in t or 'Family' in t or 'Classic' in t or 'Large' in t or 'Double' in t or 'Cosy' in t or 'Single' in t or 'Connecting' in t or 'Queen' in t or 'Cozy' in t or 'Studio' in t or 'Adjacent' in t or 'Two' in t:
                new_tags[index][idx] = 'Medium'
            
for index, i in enumerate(new_tags):
    for idx, t in enumerate(i):            
            if t not in ['Stayed 1-2 nights','Stayed 3-4 nights', 'Stayed 5+ nights', 'Fancy', 'Budget', 'Medium', 'High', ' Business trip ', ' Solo traveler ', ' Leisure trip ', ' Couple ', ' Group ', ' Family with young children ', ' Family with older children ']:
                   new_tags[index][idx] = 'High'
            
#tags_final = []
#for index, i in enumerate(new_tags):
#    new = []
#    for idx, t in enumerate(i):
#        if t in ['Stayed 1-2 nights','Stayed 3-4 nights', 'Suite', 'Apartment', 'Fancy', 'Budget', 'Medium', 'High', ' Business Trip ', ' Solo Traveler ', ' Leisure trip ', ' Couple ', ' Group ', ' Family with young children ', ' Family with older children ']:
#            new.append(i[idx])
#    tags_final.append(new)
'''

In [ ]:
unique_tags = []
for i in new_tags:
    for t in i:
        if t not in unique_tags:
            unique_tags.append(t)

In [ ]:
#Replace tags with feature engineered tags
data['New_Tags'] = new_tags

In [ ]:
#Onehotencode tags and drop unneeded columns
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

tagdf = pd.DataFrame(mlb.fit_transform(data.New_Tags),columns=mlb.classes_, index=data.index)


data = data.join(tagdf)
data = data.drop(['New_Tags'], axis=1)
data.drop('Tags', axis=1, inplace=True)

In [ ]:
neg = data.pop('Negative_Review')
pos = data.pop('Positive_Review')

In [ ]:
# didn't end up using nationality
#nat = pd.get_dummies(data['Reviewer_Nationality'], prefix='Nationality: ')
#nat = nat.astype(int)
#data = data.join(nat)
#data = data.drop(['Reviewer_Nationality'], axis=1)


In [ ]:
lat = data.pop('lat')
lng = data.pop('lng')
data = data.drop(['Reviewer_Nationality'], axis=1)

In [ ]:
#clean = clean.to_csv('clean.csv')

In [ ]:
#remove stopwords and lemmatize reviews
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

data['Neg_Review_Clean'] = data['Negative_Review']

lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

filtered = []
for i in data['Neg_Review_Clean']:
    i = i.split()
    filtered_sentence = [lem.lemmatize(w.lower()) for w in i if w not in stop_words]
    filtered.append(' '.join(filtered_sentence))
    
cleanest['Neg_Review_Clean'] = filtered

In [ ]:
#remove stopwords and lemmatize reviews
data['Pos_Review_Clean'] = data['Positive_Review']

lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

filteredpos = []
for i in data['Pos_Review_Clean']:
    i = i.split()
    filtered_sentence = [lem.lemmatize(w) for w in i if w not in stop_words]
    filteredpos.append(' '.join(filtered_sentence))
    
cleanest['Pos_Review_Clean'] = filteredpos

In [ ]:
neg = cleanest['Neg_Review_Clean']
pos = cleanest['Pos_Review_Clean']

In [ ]:
from textblob import TextBlob
neg_review_sentiment = []
for i in neg:
    review = TextBlob(i)
    neg_review_sentiment.append(review.sentiment)


In [ ]:
neg_review_sentiments = [i.polarity for i in neg_review_sentiment]
cleanest['neg_review_sentiment'] = neg_review_sentiments

In [ ]:
pos_review_sentiment = []
for i in pos:
    review = TextBlob(i)
    pos_review_sentiment.append(review.sentiment)

In [ ]:
pos_review_sentiments = [i.polarity for i in pos_review_sentiment]
cleanest['pos_review_sentiment'] = pos_review_sentiments

In [ ]:
#cleanest.to_csv('cleanest.csv')

In [ ]:
#lemmed = data.to_csv('lemmed.csv')

In [ ]:
cleanest = pd.read_csv('cleanest.csv')

In [ ]:
cleanest.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Neg_Review_Clean', 'Pos_Review_Clean'], axis=1, inplace=True)
y = cleanest.pop('Reviewer_Score')

In [ ]:
negdf = vader[(vader['neg_sent']<-.95) & (vader['pos_sent']<-.95)]

In [ ]:
negdf

In [ ]:
negative = data.loc[data['neg_review_sentiment']==-1]
posnegative = data.loc[data['pos_review_sentiment']==-1]


posnegative = data.loc[data['neg_review_sentiment']==1]
positive = data.loc[data['pos_review_sentiment']==1]


In [ ]:
print('1 neg ',data.loc[5838]['Negative_Review'])
print(data.loc[5838]['Positive_Review'])
print(data.loc[137862]['Negative_Review'])
print(data.loc[137862]['Positive_Review'])
print(data.loc[221918]['Negative_Review'])
print(data.loc[221918]['Positive_Review'])
print(data.loc[393760]['Negative_Review'])
print(data.loc[393760]['Positive_Review'])
print(data.loc[464189]['Negative_Review'])
print(data.loc[464189]['Positive_Review'])
print(data.loc[502957]['Negative_Review'])
print(data.loc[502957]['Positive_Review'])
print(data.loc[509441]['Negative_Review'])
print(data.loc[509441]['Positive_Review'])


In [ ]:
posdf = vader[(vader['neg_sent']>.99) & (vader['pos_sent']>.99)]

In [ ]:
posdf

In [ ]:
positive[positive['neg_review_sentiment']==1].sort_values('Reviewer_Score')

In [ ]:
print(data.loc[79198]['Negative_Review'])
print(data.loc[79198]['Positive_Review'])
print(data.loc[88327]['Negative_Review'])
print(data.loc[88327]['Positive_Review'])
print(data.loc[102118]['Negative_Review'])
print(data.loc[102118]['Positive_Review'])
print(data.loc[177909]['Negative_Review'])
print(data.loc[177909]['Positive_Review'])
print(data.loc[186402]['Negative_Review'])
print(data.loc[186402]['Positive_Review'])
print(data.loc[187132]['Negative_Review'])
print(data.loc[187132]['Positive_Review'])

## Modeling

In [ ]:
#modeling = data.copy()

In [ ]:
modeling.drop(['Negative_Review', 'Positive_Review', 'Neg_Review_Clean', 'Pos_Review_Clean'], axis=1, inplace=True)

In [ ]:
#modeling = modeling.to_csv('modeling.csv')

In [ ]:
modeling = pd.read_csv('modeling.csv')

In [ ]:
modeling.drop('Unnamed: 0', axis=1, inplace=True)
y = modeling.pop('Reviewer_Score')

In [ ]:
modeling.drop('days_since_review', axis=1, inplace=True)

In [ ]:
X = cleanest

In [ ]:
clean = pd.read_csv('clean.csv')
y = clean.pop('Reviewer_Score')

In [ ]:
clean.drop(['Unnamed: 0.1','Unnamed: 0'], axis=1, inplace=True)

In [ ]:
clean

#
#
#
#END CLEANING

In [ ]:
scaled = modeling.copy()

In [ ]:
scaled

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
#Scale select columns
cols_to_scale = ['Additional_Number_of_Scoring', 'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'neg_review_sentiment', 'pos_review_sentiment']

scaled[cols_to_scale] = scaler.fit_transform(scaled[cols_to_scale])



In [ ]:
#scaleddf = scaled.to_csv('scaled.csv')

In [ ]:
scaled = pd.read_csv('scaled.csv')

In [ ]:
scaled.drop('Unnamed: 0', axis=1, inplace=True)
y = data.pop('Reviewer_Score')

In [ ]:
#MODELING

In [ ]:
X = scaled

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2)

In [ ]:
scaled.shape

In [ ]:
cleanest

In [ ]:
rf = RandomForestRegressor(min_samples_leaf = 30,
                          max_depth=25,
                          max_features=10,
                          n_estimators=500)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict(X_test)

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
gdbr = GradientBoostingRegressor(learning_rate=0.01,
                                  loss='ls',
                                 max_depth=15,
                                  n_estimators=100,
                                 min_samples_leaf=120,
                                 max_features=10,
                                  random_state=1)

In [ ]:
gdbr.fit(X_train, y_train)

In [ ]:
y_hat = gdbr.predict(X_test)
mean_absolute_error(y_test, y_hat)

In [ ]:
rf = RandomForestRegressor()
param_dist = dict(min_samples_leaf = [5,10,15,20],
                          max_depth = [5,10,20,30],
                          max_features=[3,5,10,15, 20],
                          n_estimators=[50,75,100,300,500])
rand2 = RandomizedSearchCV(rf, param_dist, cv=10, scoring='neg_mean_absolute_error', n_iter=10)

In [ ]:
rand2.fit(X_train, y_train)

from pprint import pprint
pprint(rand2.best_estimator_.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 30,
 'max_features': 40,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 500,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

#Best random forest results
RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=25,
                                                   max_features=10,
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=30,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=500,
                                                   n_jobs=None, oob_score=False,
                                                   ra...rbose=0,
                                                   warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': [10, 20, 30, 40, 50],
                                        'min_samples_leaf': [5, 10, 15, 20, 30,
                                                             50],
                                        'n_estimators': [50, 75, 100, 300, 500,
                                                         1000]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='neg_mean_absolute_error',
                   verbose=0)

In [ ]:
yrand2 = rand2.predict(X_test)
mean_absolute_error(y_test, yrand2)

With cleanest
rand2.best_estimator_.get_params

<bound method BaseEstimator.get_params of RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features=15, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=10,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)>

In [ ]:
rf = RandomForestRegressor(min_samples_leaf = 5,
                          max_depth = 30,
                          max_features=5,
                          n_estimators=500)

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
#.88
yrand2 = rf.predict(X_test)
mean_absolute_error(y_test, yrand2)

In [ ]:
r2_score(y_test,yrand2)

In [ ]:
feats = rf.feature_importances_
columns = vader.columns
zipped = zip(columns, feats)
top_feats = sorted(zipped, key = lambda x: x[1], reverse=True)
top_feats

In [ ]:
top_features = sorted(zipped, key = lambda x: x[1], reverse=True) 

In [ ]:
vadercount = pd.read_csv('vadercount2.csv')

In [ ]:
vadercount.columns

In [ ]:
y

In [ ]:
vadercount.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1'], axis=1, inplace=True)

In [ ]:
y = pd.read_csv('y.csv')

In [ ]:
Xv = vadercount
y = y['Reviewer_Score']

In [ ]:
#y.to_csv('y.csv')

In [ ]:
y.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
Xv_train, Xv_test, yv_train, yv_test = train_test_split(Xv,y, test_size=.2)

In [ ]:
rf = RandomForestRegressor()
param_dist = dict(min_samples_leaf = [5,10,15,20],
                          max_depth = [5,10,20,30],
                          max_features=[3,5,10,15, 20],
                          n_estimators=[50,75,100,300,500])
rand2 = RandomizedSearchCV(rf, param_dist, cv=10, scoring='neg_mean_absolute_error', n_iter=10)

In [ ]:
rand2.fit(Xv_train, yv_train)

In [ ]:
#0.8644864926810877
yvadersearch = rand2.predict(Xv_test)
mean_absolute_error(yv_test, yvadersearch)

#Best rf
rand2.best_estimator_.get_params
<bound method BaseEstimator.get_params of RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features=15, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=10,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)>

In [ ]:
feats = rf.feature_importances_
cols = scaled.columns
zipped = zip(cols, feats)

In [ ]:
gdbr = GradientBoostingRegressor(learning_rate=0.01,
                                  loss='ls',
                                 max_depth=15,
                                  n_estimators=100,
                                 min_samples_leaf=120,
                                 max_features=10,
                                  random_state=1)

gdbr.fit(Xv_train, yv_train)

In [ ]:
#0.867067557566572
ygvader = rf.predict(Xv_test)
mean_absolute_error(yv_test, ygvader)

In [ ]:
clean

In [ ]:
gdbr = GradientBoostingRegressor()
param_dist = dict(min_samples_leaf = [10,15,20,30,50],
                          max_depth = [5,10,20,30],
                          max_features=[10, 20, 30],
                          n_estimators=[50,100,300,500],
                         learning_rate = [0.1, .05, 0.07, 0.01])
booster2 = RandomizedSearchCV(gdbr, param_dist, cv=10, scoring='neg_mean_absolute_error', n_iter=10)

In [ ]:
clean

In [ ]:
booster2.fit(X_train, y_train)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.01,
                                                       loss='ls', max_depth=15,
                                                       max_features=10,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=120,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       n_estimators=100...
                                                       warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': [0.1, 0.05, 0.07,
                                                          0.01],
                                        'max_depth': [5, 10, 20, 30],
                                        'max_features': [10, 20, 30, 40],
                                        'min_samples_leaf': [10, 15, 20, 30,
                                                             50],
                                        'n_estimators': [50, 100, 300, 500,
                                                         1000]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='neg_mean_absolute_error',
                   verbose=0)

In [ ]:
yboost2 = booster2.predict(X_test)
mean_absolute_error(y_test, yboost2)

In [ ]:

gdbr = GradientBoostingRegressor(max_depth=30, max_leaf_nodes=40, min_samples_leaf=50, n_estimators=100)

gdbr.fit(X_train, y_train)

In [ ]:
yboost2 = gdbr.predict(X_test)
mean_absolute_error(y_test, yboost2)

In [ ]:
boost_feats = gdbr.feature_importances_
columnss = modeling.columns
zipped = zip(columnss, boost_feats)
top_feats = sorted(zipped, key = lambda x: x[1], reverse=True)
top_feats[:10]

#Best gradient boost score
from pprint import pprint
pprint(booster2.best_estimator_.get_params())
{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 30,
 'max_features': 40,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 50,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [ ]:
range_of_model = y_test-yboost2

In [ ]:
#Largest Guess over
range_of_model.min()

In [ ]:
Lowest Guess Below
range_of_model.max()

In [ ]:
np.mean(range_of_model)

In [ ]:
xg_train = xgb.DMatrix(Xv_train, label=yv_train)

In [ ]:
xg_test = xgb.DMatrix(Xv_test, label=yv_test)
param = {}
# I used gamma regression 
param['objective'] = 'reg:gamma'
param['eta'] = 0.05
param['max_depth'] = 9
param['silent'] = 0
param['nthread'] = 4
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 1000

In [ ]:
bst = xgb.train(param, xg_train, num_round, watchlist)
pred = bst.predict(xg_test)

In [ ]:
#Calculate Error
error = pred - yv_test
error = abs(error)
error_rate = np.mean(error)/np.mean(yv_test)
print("error rate = %f " %error_rate )

In [ ]:
#vader .811
mean_absolute_error(yv_test,pred)

In [ ]:
'''best:
xg_test = xgb.DMatrix(Xv_test, label=yv_test)
param = {}
# I used gamma regression 
param['objective'] = 'reg:gamma'
param['eta'] = 0.05
param['max_depth'] = 9
param['silent'] = 0
param['nthread'] = 4
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 1000
'''

In [ ]:
feats_dict = bst.get_score(importance_type='gain')

In [ ]:
items = feats_dict.items()

In [ ]:
sorted(feats_dict.values())

In [ ]:
#Testing without average score
sans_average = clean
sans_average.drop('Average_Score', axis=1, inplace=True)

In [ ]:
XA_train, XA_test, y2_train, y2_test = train_test_split(sans_average, y, test_size=0.2)

In [ ]:

gdbr2 = GradientBoostingRegressor(max_depth=30, max_leaf_nodes=40, min_samples_leaf=50, n_estimators=100)

gdbr2.fit(XA_train, y2_train)

In [ ]:
yboost2A = gdbr2.predict(XA_test)
mean_absolute_error(y_test, yboost2A)

In [ ]:
r2_score(y_test, yboost2A)

In [ ]:
negative_reviews[5245]

In [ ]:
negative_words = {}
for i in negative_reviews:
    try:
        for w in i.split():
            if w not in negative_words.keys():
                negative_words[w] = 0
            negative_words[w] +=1
    except AttributeError:
        continue
        

In [ ]:
top_list = sorted(negative_words.values())[::-1][:30]
for k,v in negative_words.items():
    if v in top_list:
        print(k)

In [ ]:
positive_words = {}
for i in positive_reviews:
    try:
        for w in i.split():
            if w not in positive_words.keys():
                positive_words[w] = 0
            positive_words[w] +=1
    except AttributeError:
        continue

In [ ]:
top_list = sorted(negative_words.values())[::-1][:30]
for k,v in negative_words.items():
    if v in top_list:
        print(k)

In [ ]:
data.Reviewer_Nationality.unique()

In [ ]:
import re

In [ ]:
europe = pd.read_html('https://en.wikipedia.org/wiki/Ethnic_groups_in_Europe')

In [ ]:
euro = list(europe[4]['Country']
for idx, i in enumerate(euro):
    if not i.isalpha():
        euro[idx] = i[:-3]
    if i =='United King':
        euro[idx] = 'United Kingdom'
euro[6] = 'Bosnia and Herzegovina'

In [ ]:
euro

In [ ]:
data['Reviewer_Nationality'].unique()

In [ ]:
#euro_or_no = list(data['Reviewer_Nationality'])
#for idx, i in enumerate(euro_or_no):
#    euro_or_no[idx] = i[1:-1]
for idx,i in enumerate(euro_or_no):

    if i in list(euro):
        euro_or_no[idx] = 'European'
    else:
        euro_or_no[idx] = 'Not European'


In [ ]:
euro_or_no

In [ ]:
data['Reviewer_Nationality'] = euro_or_no

In [ ]:
euros = data.copy()

In [ ]:
euros.drop(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
        'Hotel_Name',
       'Negative_Review', 
       'Total_Number_of_Reviews', 'Positive_Review',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Tags',
       'days_since_review', 'lat', 'lng'], axis=1, inplace=True)

In [ ]:
euros.Reviewer_Nationality.unique()

In [ ]:
euros.groupby('Reviewer_Nationality').mean()

In [ ]:
negrevs[:100]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
lemmed = pd.read_csv('lemmed.csv')

In [ ]:
neg_content = lemmed['Neg_Review_Clean']
pos_content = lemmed['Pos_Review_Clean']

In [ ]:
word_tokenize(neg_content)

In [ ]:
# try ngrams 2,2
count = CountVectorizer(max_features=1000, stop_words='english', ngram_range=(2,2))

In [ ]:
fitted_neg = count.fit_transform(lemmed['Neg_Review_Clean'].values.astype('U')) 
fitted_pos = count.fit_transform(lemmed['Pos_Review_Clean'].values.astype('U'))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
tfidf_pos = tfidf.fit_transform(lemmed['Pos_Review_Clean'].values.astype('U'))


In [ ]:
tfidf_neg = tfidf.fit_transform(lemmed['Neg_Review_Clean'].values.astype('U'))

In [ ]:
tfidf_pos = tfidf_pos.toarray()

In [ ]:
tfidf_neg.toarray()

In [ ]:
count = CountVectorizer(max_features=300, stop_words='english')

In [ ]:
fitted_pos = count.fit_transform(lemmed['Pos_Review_Clean'].values.astype('U'))

In [ ]:
pos_col_names = count.get_feature_names()
fitted_pos = fitted_pos.todense()

In [ ]:
fitted_pos = pd.DataFrame(fitted_pos, columns = pos_col_names)

In [ ]:
vader3 = pd.read_csv('vader3.csv')

In [ ]:
vader3 = vader3.join(fitted_pos)

In [ ]:
vader3.to_csv('vadercount.csv')

In [ ]:
vadercount = pd.read_csv('vadercount2.csv')

In [ ]:
fitted_neg = count.fit_transform(lemmed['Neg_Review_Clean'].values.astype('U'))
neg_col_names = count.get_feature_names()
fitted_neg = fitted_neg.todense()


In [ ]:
fitted_neg = pd.DataFrame(fitted_neg, columns = neg_col_names)

In [ ]:
fitted_neg = fitted_neg.add_suffix('_neg')

In [ ]:
fitted_neg = pd.read_csv('fittedneg.csv')

In [ ]:
fitted_neg.to_csv('fittedneg.csv')

In [ ]:
fitted_neg.drop('Unnamed: 0', axis=1,inplace=True)

In [ ]:
vadercount = vadercount.join(fitted_neg)

In [ ]:
vadercount.to_csv('vadercountfinal.csv')

In [ ]:
vader3.to_csv('vadercount2.csv')

In [ ]:
vadercount = pd.read('vadercount2.csv')

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=5, init='random', random_state=0)
W = model.fit_transform(fitted_neg)
H = model.components_

In [ ]:
from sklearn.decomposition import NMF
def fit_nmf(r):
    nmf = NMF(n_components=r)
    nmf.fit(fitted_neg)
    W = nmf.transform(fitted_neg)
    H = nmf.components_
    return nmf.reconstruction_err_


fit_nmf(10)
error = [fit_nmf(i) for i in range(1,11)]
plt.plot(range(1,11), error)
plt.xticks(range(1, 11))
plt.xlabel('r')
plt.ylabel('Reconstruction Error')

In [ ]:
words = count.get_feature_names()

In [ ]:
words2 = count2.get_feature_names()

In [ ]:
def fit_nmf_pos(r):
    nmf = NMF(n_components=r)
    nmf.fit(fitted_pos)
    W = nmf.transform(fitted_pos)
    H = nmf.components_
    return nmf.reconstruction_err_

fit_nmf_pos(8)
error = [fit_nmf(i) for i in range(1,10)]
plt.plot(range(1,10), error)
plt.xticks(range(1, 10))
plt.xlabel('r')
plt.ylabel('Reconstruction Error')

In [ ]:
count3 = CountVectorizer(max_features=1000, stop_words='english', ngram_range=(3,3))

In [ ]:
words3 = count3.get_feature_names()

In [ ]:
fitted_neg3 = count3.fit_transform(neg_content.values.astype('U')) 
fitted_pos3 = count3.fit_transform(pos_content.values.astype('U'))

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=20, init='random', random_state=0)
W = model.fit_transform(fitted_neg3)
H = model.components_

In [ ]:
associated = []
for i in H:
    associated.append((np.argsort(i)[::-1][:10]))

In [ ]:
topics = []
for num, i in enumerate(associated):
    lst = []
    for idx in i:
        lst.append(words3[idx])
    topics.append(lst)
    print(f'topic{num+1} :', lst)

In [ ]:
from sklearn.decomposition import NMF
model = NMF(n_components=3, init='random', random_state=0)
W = model.fit_transform(fitted_neg)
H = model.components_

In [ ]:
associated = []
for i in H:
    associated.append((np.argsort(i)[::-1][:30]))

In [ ]:
topics = []
for num, i in enumerate(associated):
    lst = []
    for idx in i:
        lst.append(words[idx])
    topics.append(lst)
    print(f'topic{num+1} :', lst)

In [ ]:
model = NMF(n_components=20, init='random', random_state=0)
W = model.fit_transform(fitted_neg)
H = model.components_

In [ ]:
H.shape

In [ ]:
associated = []
for i in H:
    associated.append((np.argsort(i)[::-1][:10]))

In [ ]:
topics = []
for num, i in enumerate(associated):
    lst = []
    for idx in i:
        lst.append(words[idx])
    topics.append(lst)
    print(f'topic{num+1} :', lst)

In [ ]:
model = NMF(n_components=3, init='random', random_state=0)
W2 = model.fit_transform(fitted_neg)
H2 = model.components_

In [ ]:
associated_pos = []
for i in H2:
    associated_pos.append((np.argsort(i)[::-1][:20]))

In [ ]:
topics_pos = []
for num, i in enumerate(associated_pos):
    lst = []
    for idx in i:
        lst.append(words[idx])
    topics_pos.append(lst)
    print(f'topic{num+1} :', lst)

In [ ]:
from textblob import TextBlob

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

data['Neg_Review_Clean'] = data['Negative_Review']

lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

filtered = []
for i in data['Neg_Review_Clean'][:2]:
    i = i.split()
    filtered_sentence = [lem.lemmatize(w) for w in i if w not in stop_words]
    filtered.append(' '.join(filtered_sentence))

In [ ]:
data['Neg_Review_Clean'][0]

In [ ]:
datafiltered[0]

In [ ]:
from textblob import TextBlob
d = TextBlob("didn't")
d.sentiment

In [ ]:
c = TextBlob(filtered[0])
c.sentiment

In [ ]:
def hand_label_topics(H, vocabulary):
    '''
    Print the most influential words of each latent topic, and prompt the user
    to label each topic. The user should use their humanness to figure out what
    each latent topic is capturing.
    '''
    hand_labels = []
    for i, row in enumerate(H):
        top_five = np.argsort(row)[::-1][:20]
        print('topic', i)
        print('-->', ' '.join(vocabulary[top_five]))
        label = input('please label this topic: ')
        hand_labels.append(label)
        print()
    return hand_labels

In [ ]:
hand_labels = hand_label_topics(topics_pos, words)

In [ ]:
#Word Clouds
from wordcloud import WordCloud
wc = WordCloud(background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)


In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [40, 10]

# Create subplots for each Topic
for index, topic in enumerate(topics):
    wc = WordCloud(width = 1000, height = 500).generate(' '.join(topic))
    
    plt.subplot(2, 5, index+1)
    plt.imshow(wc, interpolation="quadric")
    plt.axis("off")
    #plt.title(hand_labels[index])
    
plt.show()

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color = 'white',
        max_words = 25,
        max_font_size = 30, 
        scale = 3,
    ).generate(str(data))

    fig = plt.figure(1, figsize = (10, 10))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize = 20)
        fig.subplots_adjust(top = 2.3)

    plt.imshow(wordcloud)
    plt.show()
    
# print wordcloud
show_wordcloud(topics[0])

In [ ]:
show_wordcloud(topics[1])

In [ ]:
show_wordcloud(topics[2])

In [ ]:
show_wordcloud(fitted_neg)

In [ ]:
# Checking monthly trend, consistent scoring
monthdf = data[['Review_Date','Average_Score','Reviewer_Score']]

In [ ]:
monthdf['Review_Date'] = pd.to_datetime(monthdf['Review_Date'])

In [ ]:
monthdf['Review_Date'] = monthdf['Review_Date'].apply(lambda x: x.strftime('%Y-%m'))

In [ ]:
monthdf['City'] = city

In [ ]:
monthdf

In [ ]:
monthdf = monthdf.groupby(['Review_Date', 'City']).agg({'Average_Score':'mean', 'Reviewer_Score':'mean'})

In [ ]:
monthdf.reset_index(inplace=True)

In [ ]:
monthdf.sort_values('City')

In [ ]:
mapdf.describe()

In [ ]:
#mapping
mapdf = data[['Hotel_Name', 'Average_Score', 'lat', 'lng']]
mapdf = mapdf.groupby(['Hotel_Name']).agg({'Average_Score':'mean', 'lat':'mean', 'lng':'mean'})
mapdf = mapdf.dropna()
mapdf.reset_index(inplace=True)

In [ ]:
mapdf['color'] = pd.cut(mapdf['Average_Score'], bins=[0,8.1,8.5,8.9,10], 
                              labels=['red', 'orange', 'blue', 'green'])

In [ ]:
import folium
m = folium.Map(location=[52.360576, 4.915968])

for i in range(mapdf.shape[0]):
    folium.CircleMarker([mapdf['lat'][i],mapdf['lng'][i]], radius=2,tooltip=f'{mapdf["Hotel_Name"][i]} : {mapdf["Average_Score"][i]:0.2}', color=mapdf['color'][i]).add_to(m)
m

In [ ]:
mapdf[mapdf['color']=='green'].sort_values('Average_Score', ascending=False)

In [ ]:
from folium.plugins import FloatImage


import folium
b = folium.Map(location=[52.360576, 4.915968])
# convert to (n, 2) nd-array format for heatmap
import folium.plugins as plugins
lat = np.array(mapdf['lat'])
lng = np.array(mapdf['lng'])
coords = []
for i,j in zip(lat,lng):
    coords.append((i,j))
hotels = np.array(coords)


'''steps = 20
color_map=cm.linear.BuGrYlRd.scale(0,1).to_step(steps)

gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = color_map.rgb_hex_str(1/steps*i)'''



image_file='legend.png'

FloatImage(image_file,bottom=5,left=5).add_to(b)

# plot heatmap
for i in range(mapdf.shape[0]):
    folium.CircleMarker([mapdf['lat'][i],mapdf['lng'][i]], radius=2, tooltip=f'{mapdf["Hotel_Name"][i]} : {mapdf["Average_Score"][i]:0.2}', color=mapdf['color'][i]).add_to(b)


b.add_children(plugins.HeatMap(hotels, radius=30))
b



In [ ]:
mapdata = np.array(mapdf[['lat', 'lng','Average_Score']])

In [ ]:
gradient_dict={}
for ind_val, c in zip(mapdf.index, mapdf.color):
    # Create gradient dictionary for heatmap on the fly
    gradient_dict[ind_val] = c

In [ ]:
from folium.plugins import HeatMap
hmap = folium.Map(location=[52.360576, 4.915968],control_scale = True, zoom_start=13)

hmap.add_child(HeatMap(mapdata, radius = 50, gradient={.25: 'red', .5: 'orange', .75: 'blue', .89: 'green'}))
hmap.caption = 'Test'
hmap

In [ ]:
# blank dictionary of values
# Take in reviews
# Remove Stop words and Lemmatize
# tfidf neg words
# tfidf pos words
# tfidf pos - tfidf neg
# tfidf ngrams
# get range
# some small range=0
# normalize values from - to -1 based on * rating above or below
# normalize values from + to 1
# add values to dictionary
# dictionary of values
#

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=10000, stop_words='english')
tfidf_pos = count.fit_transform(pos_content.values.astype('U'))
tfidf_neg = count.fit_transform(neg_content.values.astype('U'))

In [ ]:
cleanest

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

analyser = SentimentIntensityAnalyzer()

In [ ]:
def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(snt)))

In [ ]:
print_sentiment_scores('If I had to choose something it would be that the nice big bed was covered by two single quilts rather than a double')

In [ ]:
a = analyser.lexicon.keys()
b = analyser.lexicon.values()

In [ ]:
hotel_lexicon = {k:v for k,v in zip(a,b)}

In [ ]:
hotel_lexicon['small']

In [ ]:
snt = analyser.polarity_scores('No Positive')

In [ ]:
snt['compound']

In [ ]:
data["Negative_Review"] = data["Negative_Review"].apply(lambda x: str(x).replace("No Negative", ""))
data["Positive_Review"] = data["Positive_Review"].apply(lambda x: str(x).replace("No Positive", ""))

In [ ]:
neg = data['Negative_Review']
neg_review_sentiment = []
for i in neg:
    snt = analyser.polarity_scores(i)
    neg_review_sentiment.append(snt['compound'])


In [ ]:
pos = data['Positive_Review']
pos_review_sentiment = []
for i in pos:
    snt = analyser.polarity_scores(i)
    pos_review_sentiment.append(snt['compound'])


In [ ]:
vader

In [ ]:
cleanest.info()

In [ ]:
cleanest = pd.read_csv('cleanest.csv')

In [ ]:
cleanest["Neg_Review_Clean"] = cleanest["Neg_Review_Clean"].apply(lambda x: str(x).replace("No Negative", ""))
cleanest["Pos_Review_Clean"] = cleanest["Pos_Review_Clean"].apply(lambda x: str(x).replace("No Positive", ""))
                                                                  

In [ ]:
cleanest

In [ ]:
vader = cleanest.copy()

In [ ]:
#vader.drop(['neg_review_sentiment', 'pos_review_sentiment'], axis=1, inplace=True)

vader['pos_sent'] = pos_review_sentiment
vader['neg_sent'] = neg_review_sentiment

In [ ]:
vader.drop(['Unnamed: 0', 'Unnamed: 0.1','Neg_Review_Clean', 'Pos_Review_Clean'], axis=1, inplace=True)

In [ ]:
vader.to_csv('vader.csv')

In [ ]:
vader.drop('Additional_Number_of_Scoring', axis=1, inplace=True)

In [ ]:
vader[vader['pos_sent']<-0.5]

In [ ]:
vader[vader['neg_sent']>0]

In [ ]:
data['Positive_Review'][577]

In [ ]:
vader

In [ ]:
data['Hotel_Address'][455551]

In [ ]:
city = data['Hotel_Address']
for idx, i in enumerate(city):
    if 'Amsterdam' in i:
        city[idx] = 'Amsterdam'
    if 'Vienna' in i:
        city[idx] = 'Vienna'
    if 'Milan' in i:
        city[idx] = 'Milan'
    if 'Barcelona' in i:
        city[idx] = 'Barcelona'
    if 'Paris' in i:
        city[idx] = 'Paris'
    if 'London' in i:
        city[idx] = 'London'
        

In [ ]:
city = pd.Series(city)

In [ ]:
vader['City'] = city

In [ ]:
cities = pd.get_dummies(vader['City'])
vader = vader.join(cities)
vader.drop(['City'], axis=1, inplace=True)

In [ ]:
vader

In [ ]:
data.head()

In [ ]:
from sklearn.naive_bayes import GaussianNB 
NB = GaussianNB()
neg_pos = data['Reviewer_Score'] > 7.25
sentiment_predictor = NB.fit(tfidf_pos, neg_pos)
sentiment_predictor.

In [ ]:
plt.figure(figsize=(12,8))
plt.hist(data['Reviewer_Score'], bins = 40)
plt.xlabel('Reviewer Score')
plt.ylabel('Occurrences')
plt.title('Frequency Distribution of Reviewer Scores')

In [ ]:
data.drop('Additional_Number_of_Scoring', axis=1, inplace=True)


In [ ]:
data.drop(['lat', 'lng', 'Total_Number_of_Reviews', 'Total_Number_of_Reviews_Reviewer_Has_Given'], axis=1, inplace=True)
data.describe()

In [ ]:
data['Reviewer_Score'].std()

In [ ]:
data['Reviewer_Score'].mean()

In [ ]:
8.395-3*1.637

In [ ]:
a = data['Reviewer_Score']<3
a.sum()

In [ ]:
lemmed = pd.read_csv('lemmed.csv')

In [ ]:
lemmed["Negative_Review"] = lemmed["Negative_Review"].apply(lambda x: str(x).replace("No Negative", ""))
lemmed["Positive_Review"] = lemmed["Positive_Review"].apply(lambda x: str(x).replace("No Positive", ""))
lemmed["Neg_Review_Clean"] = lemmed["Neg_Review_Clean"].apply(lambda x: str(x).replace("No Negative", ""))
lemmed["Pos_Review_Clean"] = lemmed["Pos_Review_Clean"].apply(lambda x: str(x).replace("No Positive", ""))

In [ ]:
lemmed

In [ ]:
lemmed["Pos_Review_Clean"][3]

In [ ]:
spell = SpellChecker()
sp = []
for i in lemmed['Pos_Review_Clean']:
    misspelled = spell.unknown(word_tokenize(i))
    for w in misspelled:
        sp.append(w)

In [ ]:
sorted(set(sp))

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')

In [ ]:
lemmed['differential'] = lemmed['Reviewer_Score'] - data['Average_Score']

In [ ]:
neg_dict = {}
for idx, i in enumerate(lemmed['Neg_Review_Clean']):
    try:
        for w in word_tokenize(i):
            if w.isalpha():
                w=w.lower()
                if w not in neg_dict.keys():
                    neg_dict[w] = 0
                neg_dict[w] -=1
    except TypeError:
        continue

In [ ]:
sorted(neg_dict.values())

In [ ]:
pos_dict = {}
for i in lemmed['Pos_Review_Clean']:
    try:
        for w in word_tokenize(i):
            w=w.lower()
            if w.isalpha():
                if w not in pos_dict.keys():
                    pos_dict[w] = 0

                pos_dict[w]+=1
    except TypeError:
        continue            


In [ ]:
sorted(pos_dict.values())[::-1]

In [ ]:
len(pos_dict.keys())

In [ ]:
len(neg_dict.keys())

In [ ]:
merged_dict = neg_dict.copy()

for i in pos_dict.keys():
    if i not in merged_dict.keys():
        merged_dict[i]=0
    merged_dict[i] += pos_dict[i]

In [ ]:
top_neg = sorted(merged_dict.items(), key=(lambda item: item[1]))[:55]

In [ ]:
top_pos = sorted(merged_dict.items(), key=(lambda item: item[1]))[73480:]

In [ ]:
#top_pos[::-1]

In [ ]:
a = analyser.lexicon.keys()
b = analyser.lexicon.values()

In [ ]:
hotel_lexicon = {k:v for k,v in zip(a,b)}

In [ ]:
in_lexicon = []
for i in top_pos:
    if i[0] in hotel_lexicon.keys():
        in_lexicon.append((i[0], hotel_lexicon[i[0]]))
    else:
        in_lexicon.append(i)

In [ ]:
pos_in_lexicon = [i for i in in_lexicon if i[1]>4]

In [ ]:
nin_lexicon = []
for i in top_neg:
    if i[0] in hotel_lexicon.keys():
        nin_lexicon.append((i[0], hotel_lexicon[i[0]]))
    else:
        nin_lexicon.append(i)

In [ ]:
neg_in_lexicon = [i for i in nin_lexicon if i[1]<-4]

In [ ]:
neg_in_lexicon 

In [ ]:
w_pos_in_lexicon = [i[0] for i in pos_in_lexicon]
w_neg_in_lexicon = [i[0] for i in neg_in_lexicon]

In [ ]:
#w_neg_in_lexicon

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
#vader3
'''pos_words = {'large':1.5,
 'minute':1,
 'convenient':1.5,
 'walking':1,
 'near':1,
 'central':1,
 'view':1,
 'walk':1,
 'spacious':2,
 'modern':2,
 'everything':1,
 'quiet':1,
 'comfy':1,
 'close':1,
 'location':1.5}

neg_words = {
 'small': -2,
 'little':-2,
 'expensive': -2,
 'work':-1,
 'not':-1,
 'air':-0.5,
 'noise': -2,
 'told':-0.5,
 'bathroom':-1,
 'water':-1,
 'booking': -1,
 'hot':-1,
 'shower':-2,
 'cold': -3,
 'price':-1,
 'slow': -2,
 'booked':-1,
 'old':-1,
 'paid':-0.5,
 'toilet':-1,
 'tiny':-2,
 'working':-1}
'''

#vader4
pos_words = {'large':1.5,
 'minute':1,
 'convenient':1.5,
 'walking':1,
 'near':1,
 'central':1,
 'view':1,
 'walk':1,
 'spacious':2,
 'modern':2,
 'everything':1,
 'quiet':1,
 'comfy':1,
 'close':1,
 'location':1.5}

neg_words = {
 'i':-.05
 'small': -2,
 'little':-2,
 'expensive': -2,
 'work':-1,
 'not':-1,
 'air':-1,
 'noise': -2,
 'told':-0.5,
 'bathroom':-1,
 'water':-1,
 'booking': -1,
 'hot':-1,
 'shower':-2,
 'cold': -3,
 'price':-1,
 'slow': -2,
 'booked':-1,
 'old':-1,
 'paid':-0.5,
 'toilet':-1,
 'tiny':-2,
 'working':-1}
analyser = SentimentIntensityAnalyzer()

analyser.lexicon.update(pos_words)
analyser.lexicon.update(neg_words)

In [ ]:
vader = pd.read_csv('vader.csv')

In [ ]:
data["Negative_Review"] = data["Negative_Review"].apply(lambda x: str(x).replace("No Negative", ""))
data["Positive_Review"] = data["Positive_Review"].apply(lambda x: str(x).replace("No Positive", ""))

In [ ]:
pos = data['Positive_Review']
pos_review_sentiment = []
for i in pos:
    snt = analyser.polarity_scores(i)
    pos_review_sentiment.append(snt['compound'])


In [ ]:
neg = data['Negative_Review']
neg_review_sentiment = []
for i in neg:
    snt = analyser.polarity_scores(i)
    neg_review_sentiment.append(snt['compound'])

In [ ]:
vader3 = vader2.copy()
vader3['new_pos_sent'] = pos_review_sentiment
vader3['New_neg_sent'] = neg_review_sentiment

In [ ]:
vader3.to_csv('vader3.csv')

In [ ]:
vader2.head()

In [ ]:
vader2.to_csv('vader2.csv')

In [ ]:
vader2 = pd.read_csv('vader2.csv')

In [ ]:
vader2.drop(['pos_sent', 'neg_sent'],axis=1, inplace=True)

In [ ]:
#y = vader3.pop('Reviewer_Score')
#Xv2 = vader3
Xv2_train, Xv2_test, yv2_train, yv2_test = train_test_split(Xv,y, test_size=.2)

In [ ]:
rf = RandomForestRegressor(max_depth=20, max_features=15, min_samples_leaf=10,
                      n_estimators=500,)

In [ ]:
#0.8644864926810877 vader, vader2: 0.8652860490218461, v3: 0.8591603772526949

rf.fit(Xv_train, yv2_train)

yvadersearch = rf.predict(Xv2_test)
mean_absolute_error(yv2_test, yvadersearch)

In [ ]:
rf = RandomForestRegressor(max_depth=50, max_features=70, min_samples_leaf=30,
                      n_estimators=500,)

In [ ]:
rf.fit(Xv_train, yv_train)

yvadersearch = rf.predict(Xv_test)
mean_absolute_error(yv_test, yvadersearch)

In [ ]:
resid = (yvadersearch-yv_test)

In [ ]:
max(yv_test)

In [ ]:
difference

In [ ]:
print(f'{np.mean(difference)})
print(min(difference)) #max overguess
print(max(difference)) # max underguess

In [ ]:
plt.bar(yv_test,resid)

In [ ]:
len(edited_dict.items())

In [ ]:
sorted(edited_dict.items(), key=(lambda item: item[1]))

In [ ]:
sent_dict = {key:val for key, val in merged_dict.items() if key in edited_dict.keys()}

In [ ]:
sent_dict

In [ ]:
data['Negative_Review'][0]

In [ ]:
class Hotel_Sentiment_Analyzer:
    from sklearn.feature_extraction.text import TfidfVectorizer
    '''Hotel Sentiment Analyzer class
    Fit takes in X an array of reviews as strings '''
    
    def __init__(self):
        pass
    
    def fit(self, X, y):
        tfidf = TfidfVectorizer(max_features=2000, stop_words='english')
        tfidf_pos = count.fit_transform(pos_content.values.astype('U'))
        tfidf_neg = count.fit_transform(neg_content.values.astype('U'))
    
    def predict(self, X):
        pass

In [ ]:
data.head()

In [ ]:
lst = [('Location', 182356),
 ('Staff', 156235),
 ('Good', 91507),
 ('Friendly', 80715),
 ('Helpful', 71799),
 ('Excellent', 59915),
 ('Nice', 59561),
 ('Clean', 58436),
 ('Comfortable', 54418),
 ('Hotel', 49731)]
words = []
vals = []
for i in lst:
    words.append(i[0])
    vals.append(i[1])


In [ ]:
pos_words = pd.DataFrame([words,vals])

In [ ]:
pos_words=pos_words.T

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.bar(pos_words[0], pos_words[1])
ax.xaxis.set_tick_params(labelsize=16, rotation=45, )
ax.set_xlabel('Positive Words')
ax.set_ylabel('Relative Occurrences')
fig.suptitle('Top 10 Positive Words Relative Occurrences', fontsize=20);

In [ ]:
neglst = [ 
('I', 54831),
 ('Small', 38226),
 ('Nothing', 32126),
 ('Room', 29869),
 ('Bit', 23675),
 ('Could', 22721),
 ('Poor', 15300),
 ('Little', 15107),
 ('Expensive', 14225),
 ('Noisy', 12803)
]
words = []
vals = []
for i in neglst:
    words.append(i[0])
    vals.append(i[1])

In [ ]:
neg_words = pd.DataFrame([words,vals])
neg_words=neg_words.T

In [ ]:
neg_words

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.bar(neg_words[0], neg_words[1])
ax.xaxis.set_tick_params(labelsize=16, rotation=45)
ax.set_xlabel('Negative Words')
ax.set_ylabel('Relative Occurrences')
fig.suptitle('Top 10 Negative Words Relative Occurrences', fontsize=20);

In [ ]:
featurelist = [('fabulous', 0.29964397592170217), ('room', 0.30194291249199645), ('Review_Total_Positive_Word_Counts', 0.3052570176688559), ('view_neg', 0.3065943580036194), ('customer_neg', 0.309148504285848), ('like', 0.3097000750242623), ('cleaned_neg', 0.3138296421587767), ('far_neg', 0.31415213928598024), ('experience_neg', 0.3160898314851383), ('upgraded', 0.31808503933448273), ('uncomfortable_neg', 0.3181067243519525), ('amazing', 0.32032346693149794), ('positive', 0.3268533664683133), ('friendly_neg', 0.3279204781542303), ('star', 0.3316088215976614), ('money', 0.33323600792235114), ('furniture_neg', 0.33556941761333337), ('thing_neg', 0.3359991773269183), ('basic_neg', 0.3371390389734257), ('worth_neg', 0.34183704961622896), ('bad_neg', 0.35738071688622886), ('excellent', 0.3594808207557529), ('service', 0.359927338350747), ('good', 0.36450212979652796), ('value_neg', 0.36939321877731096), ('Average_Score', 0.37155553164158), ('maybe_neg', 0.3742655712045161), ('slightly_neg', 0.3756614262933333), ('problem_neg', 0.37852534112012903), ('negative_neg', 0.3795259726073531), ('liked_neg', 0.3798875420290196), ('loved', 0.38373564215679484), (' Leisure trip ', 0.3845988736897397), ('thing', 0.3854006269861976), ('tiny_neg', 0.3855309579764056), ('smell_neg', 0.39104875876111), ('comfort', 0.396663971459846), ('service_neg', 0.40567563202398277), ('poor_neg', 0.40716188873969195), ('location', 0.4366442383734017), ('exceptional', 0.44605379501967746), ('fantastic', 0.4506617635248613), ('manager_neg', 0.4539769613296106), ('dated_neg', 0.4552046506772193), ('little_neg', 0.4669997007938732), ('bed_neg', 0.48724146197141455), ('location_neg', 0.5196409792334291), ('terrible_neg', 0.5253355840173783), ('ok', 0.533507276280509), ('old_neg', 0.5588221246639997), ('rude_neg', 0.5628752955510673), ('bit_neg', 0.5665008095155175), ('overpriced_neg', 0.5972159823682486), ('staff', 0.6016395407762328), ('hotel_neg', 0.6126645258793607), ('clean_neg', 0.6272994880442819), ('rooms_neg', 0.6394214532205174), ('Review_Total_Negative_Word_Counts', 0.643662427137203), ('star_neg', 0.6624058946175011), ('money_neg', 0.6654582424585267), ('cleanliness', 0.702056108507394), ('perfect_neg', 0.7507870704583097), ('fault_neg', 0.7786412114249133), ('dirty_neg', 0.9736228654579733), ('staff_neg', 1.1175715739400547), ('new_pos_sent', 1.197688207777768), ('New_neg_sent', 1.3588873831895074), ('room_neg', 3.213848362698663)][::-1]

In [ ]:
featurelist